In [ ]:
##this script is used for extracting the ms2 data from the targetd msms data and used for spectrum comparison.

In [6]:
#helper functions
# !pip install pyopenms
# !pip install numpy
# !pip install tabulate
# !pip install matchms
import re
import numpy as np
import matchms 
from matchms import calculate_scores
from matchms import Spectrum
from matchms.similarity import CosineGreedy
import pandas as pd
import pyopenms as oms
import tabulate
import os 
import copy
import warnings
import logging
# Suppress specific warnings
warnings.filterwarnings("ignore", message=".*")

def ppm_error(mz, target):
    return abs(mz - target) / target * 1e6

def preprocess_spectra(filepath, target_mz, ce_level, tolerance=5):
    # Extract spectra with specified collision energy level for target m/z 
    if target_mz is None:
        raise ValueError("Please provide a target m/z value.")
    ce_level = float(ce_level)
    tolerance = float(tolerance)
    target_mz = float(target_mz)
    spectra = oms.MSExperiment()
    oms.MzMLFile().load(filepath, spectra)
    ms2_spec = spectra.getSpectra()

    ms2_query = oms.MSExperiment()
    for s in ms2_spec:
        if s.getMSLevel() == 1:
            continue
        precursors = s.getPrecursors()
        mslevel = s.getMSLevel()
        collision_energy = precursors[0].getMetaValue("collision energy")
        ms_error = ppm_error(precursors[0].getMZ(), target_mz)
        if ms_error < tolerance and mslevel == 2 and collision_energy == ce_level:
            ms2_query.addSpectrum(s)
        else :
            continue

    ms2beforemerg = [s for s in ms2_query.getSpectra() if s.getMSLevel() == 2]

    # print(f'Number of MS2 spectra before merge: {len(ms2beforemerg)}')

    if len(ms2beforemerg) == 0:
        # print(f"No spectra found with specified parameters for {ce_level}.")
        return None
    
    elif len(ms2beforemerg) > 1:
        #merget the ms spectra
        merger = oms.SpectraMerger()
        param = merger.getParameters()
        param.setValue("mz_tolerance", 1e-3)
        param.setValue("rt_tolerance", "5.0")
        merger.setParameters(param)
        merger.mergeSpectraPrecursors(ms2_query)
        return ms2_query
    elif len(ms2beforemerg) == 1:
        return ms2_query
    


def parse_msp_file(msp_file_path, polarity):
    """
    Parses the .msp file to extract all spectrum information.
    
    Args:
        msp_file_path (str): Path to the .msp file.

    Returns:
        dict: A dictionary where keys are InChIKey or SMILES and values are lists of spectra data.
    """
    spectra = {}
    if polarity == "positive":
        p_type = "[M+H]+"
    elif polarity == "negative":
        p_type = "[M-H]-"
    else:
        raise ValueError("Invalid polarity. Please specify 'positive' or 'negative'.")
    
    with open(msp_file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        entries = content.split('Name: ')
        for entry in entries[1:]:
            lines = entry.strip().split('\n')
            metadata = {}
            spectrum_data = []
            key = None

            for line in lines:
                if line.startswith("InChIKey:"):
                    key = line.split(": ")[1].strip()
                elif line.startswith("SMILES:") and key is None:
                    key = line.split(": ")[1].strip()
                elif line.startswith("Spectrum_type:"):
                    spectrum_type = line.split(": ")[1].strip()
                elif line.startswith("Precursor_type:"):
                    precursor_type = line.split(": ")[1].strip()
                elif line.startswith("Num Peaks:"):
                    num_peaks = int(line.split(": ")[1].strip())
                    spectrum_data = lines[lines.index(line)+1:lines.index(line)+1+num_peaks]
                elif ": " in line:
                    k, v = line.split(": ", 1)
                    metadata[k.strip()] = v.strip()

            # Retain only spectra with "Spectrum_type: MS2"
            if spectrum_type == "MS2" and precursor_type == p_type  and key and spectrum_data:
                if key not in spectra:
                    spectra[key] = []
                spectra[key].append({
                    "metadata": metadata,
                    "spectrum": [(float(mz), float(intensity)) for mz, intensity in (line.split() for line in spectrum_data)]
                })

    return spectra


def parse_mona_database(db_file_path, polarity):
    """
    Parses an alternate database format to extract all spectrum information based on polarity.

    Args:
        db_file_path (str): Path to the database file.
        polarity (str): The ion polarity to filter ("positive" or "negative").

    Returns:
        dict: A dictionary where keys are InChIKey or SMILES and values are lists of spectra data.
    """
    spectra = {}

    if polarity == "positive":
        p_type = "[M+H]+"
    elif polarity == "negative":
        p_type = "[M-H]-"
    else:
        raise ValueError("Invalid polarity. Please specify 'positive' or 'negative'.")

    with open(db_file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        entries = content.split('NAME: ')  # "NAME:" marks the beginning of each spectrum.
        for entry in entries[1:]:
            lines = entry.strip().split('\n')
            metadata = {}
            spectrum_data = []
            key = None

            for line in lines:
                if line.startswith("INCHIKEY:"):
                    key = line.split(": ")[1].strip()
                elif line.startswith("SMILES:") and key is None:
                    key = line.split(": ")[1].strip()
                elif line.startswith("PRECURSORTYPE:"):
                    ionization = line.split(": ")[1].strip()
                elif line.startswith("Num Peaks:"):
                    num_peaks = int(line.split(": ")[1].strip())
                    spectrum_data = [tuple(map(float, peak.split())) for peak in lines[lines.index(line)+1:] if peak.strip()]
                elif ": " in line:
                    k, v = line.split(": ", 1)
                    metadata[k.strip()] = v.strip()

            # Retain only spectra matching the desired polarity
            if ionization == p_type and key and spectrum_data:
                if key not in spectra:
                    spectra[key] = []
                spectra[key].append({
                    "metadata": metadata,
                    "spectrum": spectrum_data
                })

    return spectra


def get_spectrum_by_key(spectra, key):
    """
    Retrieves all spectrum information by InChIKey or SMILES.

    Args:
        spectra (dict): Parsed spectra data.
        key (str): The InChIKey or SMILES to search for.

    Returns:
        dict: A dictionary where keys are collision energies and values are spectra.
    """
    if key not in spectra:
        return {}

    grouped_spectra = {}
    for spec in spectra[key]:
        collision_energy = spec["metadata"].get("Collision_energy", spec["metadata"].get("COLLISIONENERGY", "Unknown"))
        if collision_energy not in grouped_spectra:
            grouped_spectra[collision_energy] = []
        grouped_spectra[collision_energy].append(spec["spectrum"])

    return grouped_spectra

def normalize_and_filter(spectrum, baseline):
    try:
        mz, intensity = spectrum.get_peaks()
    except AttributeError: 
        if isinstance(spectrum, list) and all(isinstance(i, tuple) for i in spectrum):
            mz, intensity = zip(*spectrum)
            mz = np.array(mz)
            intensity = np.array(intensity)
        else:
            raise ValueError("Spectrum format is not recognized.")
    except:
        if isinstance(spectrum, tuple) and len(spectrum) == 2:
            mz = np.array(spectrum[0])
            intensity = np.array(spectrum[1])
        else:
            raise ValueError("Spectrum format is not recognized.")
    
    # select the top 8 most intense peaks
    if len(intensity) > 8:
        top_indices = np.argsort(intensity)[-8:]
        mz_res = mz[top_indices]
        int_res = intensity[top_indices]
    else:
        top_indices = np.argsort(intensity)
        mz_res = mz[top_indices]
        int_res = intensity[top_indices]

    # Assert that the length of mz and intensity are the same
    if len(mz_res) != len(int_res):
        raise ValueError("The length of mz and intensity arrays do not match after filtering.")
    
    # Sort mz values in ascending order and reorder intensities correspondingly.
    sorted_indices = np.argsort(mz_res)
    mz_res = mz_res[sorted_indices]
    int_res = int_res[sorted_indices]
    return mz_res, int_res


def update_RTMSMS_level(data):
    for iter, row in data.iterrows():
        #take the maximum value of the library_best_match_score_value and the library_best_match_score_value2
        library_score = row['library_best_match_score_value']
        insilico_score = row['insilico_best_match_score_value']
        RTMS_level = row['annotation_RTMS']

        #RTMS_level 
        if library_score >=0.5:
            if RTMS_level =='NotMatched':
                data.loc[iter, 'annotation_level'] = 'NotMatched'
            elif RTMS_level == '2':
                data.loc[iter,'annotation_level'] = '2a'
            elif RTMS_level == '3':
                data.loc[iter, 'annotation_level'] = '2b'
            elif RTMS_level == '4':
                data.loc[iter,'annotation_level'] = '3'
        elif library_score <0.5 and insilico_score >=0.5:
            if RTMS_level == 'NotMatched':
                data.loc[iter, 'annotation_level'] = 'NotMatched'
            elif RTMS_level =='2' or RTMS_level == '3':
                data.loc[iter, 'annotation_level'] = '2b'
            elif RTMS_level =='4':
                data.loc[iter, 'annotation_level'] = '3'
        elif library_score <0.5 and library_score >0 and insilico_score > 0 and insilico_score < 0.5:
            if RTMS_level =='NotMatched':
                data.loc[iter, 'annotation_level'] = 'NotMatched'
            else:
                data.loc[iter, 'annotation_level'] = '4'
        elif library_score == 0 and insilico_score == 0:
            if RTMS_level =='NotMatched':
                data.loc[iter, 'annotation_level'] = 'NotMatched'
            else:
                data.loc[iter,'annotation_level'] = '5'
        else:
            if RTMS_level == 'NotMatched':
                data.loc[iter, 'annotation_level'] = 'NotMatched'
            else:
                data.loc[iter, 'annotation_level'] = '5'
        
        #check if best_score column exists
        if 'best_score' in row and row['best_score'] is not None:
            best_score = row['best_score']
            if best_score >= 0.5:
                data.loc[iter, 'annotation_level'] = '1'
            elif best_score <= 0.3 and best_score >= 0:
                data.loc[iter, 'annotation_level'] = 'NotMatched'
            else:
                continue

    return data

#remove redundant rows by comparing spectrum scores for positive and negative data based on the DTXSID
def remove_redundant_rows(df):
    # Ensure numeric columns for comparison
    df['library_best_match_score_value'] = pd.to_numeric(df['library_best_match_score_value'], errors='coerce')
    df['row_median_peak_intensity'] = df.filter(like='BH').median(axis=1)

    # Subset rows with unique DTXSID
    df_unique = pd.DataFrame()
    unique_dtxsid = df['DTXSID'].unique()

    for dtxsid in unique_dtxsid:
        subset = df[df['DTXSID'] == dtxsid]
        # If there are multiple rows for the same DTXSID
        if len(subset) > 1:
            # Select rows with the highest library_best_match_score_value
            subset2 = subset[subset['library_best_match_score_value'] == subset['library_best_match_score_value'].max()]
            if len(subset2) > 1:
                # Select rows with the highest median intensity
                subset3 = subset2[subset2['row_median_peak_intensity'] == subset2['row_median_peak_intensity'].max()]
                df_unique = pd.concat([df_unique, subset3], axis=0, ignore_index=True)
            else:
                df_unique = pd.concat([df_unique, subset2], axis=0, ignore_index=True)
        else:
            # Concatenate the rows and move to the next DTXSID
            df_unique = pd.concat([df_unique, subset], axis=0, ignore_index=True)

    return df_unique


def find_best_match(references, queries, tolerance, key):
    """
    Calculate cosine‐greedy scores and return the best match info,
    or None for everything if no references or on error.
    """
    if not references:
        return None, None, None, None

    try:
        scores = matchms.calculate_scores(
            references=references,
            queries=queries,
            similarity_function=CosineGreedy(tolerance=tolerance),
            is_symmetric=False
        )

        best_scores, num_match_peaks, best_refs, best_queries = [], [], [], []
        for query in queries:
            matches = scores.scores_by_query(query, 'CosineGreedy_score', sort=True)
            for ref, (score, n_peaks) in matches:
                best_scores.append(score)
                num_match_peaks.append(n_peaks)
                best_refs.append(ref.metadata['peak_comments'])
                best_queries.append(query.metadata['peak_comments'])

        if not best_scores:
            return None, None, None, None
        
        idx = best_scores.index(max(best_scores))
        return (
            best_scores[idx],
            num_match_peaks[idx],
            best_refs[idx],
            best_queries[idx],
        )

    except (IndexError, ValueError):
        # print(f'No match found for compound {key}')
        return None, None, None, None
    
def update_RTMSMS_level2(df):
    #update level 1
    df.loc[(df['best_score'] >= 0.5) & (df['best_spectrum_query'].notnull()), 'annotation_level'] = '1'
    df.loc[(df['best_score'] <= 0.3), 'annotation_level'] = 'NotMatched'
    
    #find rows with annotation level == 2a and 2b
    df_rest = df[(df['annotation_level'] != '2a') & (df['annotation_level'] != '2b')]
    df = df[(df['annotation_level'] == '2a') | (df['annotation_level'] == '2b')]
    #update annotation level to 3 if the row with the library_best_match_num_matchpeak < 2.0
    df.loc[(df['library_best_match_num_matchpeak'] < 2.0) & (df['annotation_level'] == '2a'), 'annotation_level'] = '4'
    df.loc[(df['library_best_match_num_matchpeak'] < 2.0) & (df['annotation_level'] == '2b'), 'annotation_level'] = '4'

    #update annotation level to 3 if the row with the insilico_best_match_num_matchpeak < 2.0
    df.loc[(df['insilico_best_match_num_matchpeak'] < 2.0) & (df['annotation_level'] == '3'), 'annotation_level'] = '4'
    df.loc[(df['insilico_best_match_num_matchpeak'] < 2.0) & (df['annotation_level'] == '3'), 'annotation_level'] = '4'
    df = pd.concat([df, df_rest], axis=0, ignore_index=True)
    return df

##remove redundant rows by steps:
#1. find rows with same DTXSID and ppm <=5 and rtfilter <=0.1: ppm = abs(mz1-mz2)/mz1*1e6, rtfilter = abs(rt1-rt2) <= 0.1
#2. find the row with the highest score and keep it, remove the rest of the rows with same DTXSID and ppm <=5 and rtfilter <=0.1

def remove_redundant_rows2(df):
    # Create a new DataFrame to store the filtered rows
    filtered_df = pd.DataFrame(columns=df.columns)
    # caculate the row median value for column starts with 'BH'
    df['row_median_intensity'] = df.filter(like='BH').median(axis=1)

    # Iterate through each row in the original DataFrame
    for index, row in df.iterrows():
        # Calculate the ppm and rtfilter values for the current row
        ppm = abs(row['Average Mz'] - df['Average Mz']) / row['Average Mz'] * 1e6
        rtfilter = abs(row['Average Rt(min)'] - df['Average Rt(min)'])

        # Find rows with same DTXSID and ppm <= 5 and rtfilter <= 0.1
        matching_rows = df[(df['DTXSID'] == row['DTXSID']) & (ppm <= 10) & (rtfilter <= 0.6)]

        # If there are matching rows, find the one with the highest score
        if not matching_rows.empty:
            try:
                #rank the mathing rows based on the annotation level
                matching_rows['annotation_level_rank'] = matching_rows['annotation_level'].map({ '1': 1, '2a': 2, '2b': 3, '3': 4, '4': 5,'5':6,'NotMatched':7})
                #sort the matching rows based on the annotation level
                matching_rows = matching_rows.sort_values(by='annotation_level_rank', ascending=True)
                #take the first row with the highest annotation level, like 1 is the highest and 5 is the lowest
                best_row = matching_rows.iloc[0]
                filtered_df = pd.concat([filtered_df, best_row.to_frame().T], ignore_index=True)
            except ValueError:
                matching_rows['best_score'] = matching_rows['best_score'].astype(float)
                best_row = matching_rows.loc[matching_rows['best_score'].idxmax()]
                filtered_df = pd.concat([filtered_df, best_row.to_frame().T], ignore_index=True)
            except ValueError:
                matching_rows['library_best_match_score_value'] = matching_rows['library_best_match_score_value'].astype(float)
                best_row = matching_rows.loc[matching_rows['library_best_match_score_value'].idxmax()]
                filtered_df = pd.concat([filtered_df, best_row.to_frame().T], ignore_index=True)
            except KeyError:
                matching_rows['insilico_best_match_score_value'] = matching_rows['insilico_best_match_score_value'].astype(float)
                best_row = matching_rows.loc[matching_rows['insilico_best_match_score_value'].idxmax()]
                filtered_df = pd.concat([filtered_df, best_row.to_frame().T], ignore_index=True)
            except Exception as e:
                #take the rows with highest row median value for the rest of the rows
                matching_rows['row_median_intensity'] = matching_rows['row_median_intensity'].astype(float)
                best_row = matching_rows.loc[matching_rows['row_median_intensity'].idxmax()]
                filtered_df = pd.concat([filtered_df, best_row.to_frame().T], ignore_index=True)          
        else:
            # If no matching rows, keep the current row
            filtered_df = pd.concat([filtered_df, row.to_frame().T], ignore_index=True)

    #make tempory column as a unique identifier for each row
    filtered_df['temp_id'] = filtered_df['DTXSID'] + '_' + filtered_df['Average Mz'].astype(str) + '_' + filtered_df['Average Rt(min)'].astype(str)
    #remove the dulipicate rows based on the temp_id column
    filtered_df = filtered_df.drop_duplicates(subset=['temp_id'], keep='first')
    #drop the temp_id column
    filtered_df = filtered_df.drop(columns=['temp_id'])

    return filtered_df

In [ ]:
##library and insilico predicted spectrum matching

In [8]:
##prepare database for spectrum matching
database1 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/MassBank_NIST.msp"
database2 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/MSMS_Public_EXP_NEG_VS17.msp"
database10 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/MSMS_Public_EXP_POS_VS17.msp"
database3 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/MoNA-export-All_LC-MS-MS_Orbitrap.msp"

#CFMID database
database4 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/CFMID-plastic/compiled_spectrum_library_neg_energy0.msp"
database5 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/CFMID-plastic/compiled_spectrum_library_neg_energy1.msp"
database6 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/CFMID-plastic/compiled_spectrum_library_neg_energy2.msp"

database7 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/CFMID-plastic/compiled_spectrum_library_pos_energy0.msp"
database8 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/CFMID-plastic/compiled_spectrum_library_pos_energy1.msp"
database9 = "D:/UCSF_postdoc_topic/REVEAL_topics/Spectrum_database/CFMID-plastic/compiled_spectrum_library_pos_energy2.msp"

mbank_data_neg = parse_msp_file(database1, polarity="negative")
mona_data_neg = parse_mona_database(database2, polarity="negative")
mona_obtrap_data_neg = parse_msp_file(database3, polarity="negative")
cfmid_e0_data_neg = parse_mona_database(database4, polarity="negative")
cfmid_e1_data_neg = parse_mona_database(database5, polarity="negative")
cfmid_e2_data_neg = parse_mona_database(database6, polarity="negative")

mbank_data_pos = parse_msp_file(database1, polarity="positive")
mona_data_pos = parse_mona_database(database10, polarity="positive")
mona_obtrap_data_pos = parse_msp_file(database3, polarity="positive")
cfmid_e0_data_pos = parse_mona_database(database7, polarity="positive")
cfmid_e1_data_pos = parse_mona_database(database8, polarity="positive")
cfmid_e2_data_pos = parse_mona_database(database9, polarity="positive")

In [286]:
#step1 extract the spectra from the database for each inquiry compound
#step2 extract the experimental spectrum from standard msms data for each inquiry compound
#step3 calculate the similarity score for each inquiry compound
#step4 save the similarity score for each inquiry compound from each database
#step5 report the results

##import the list of suspected compounds for spectrum match
negfilepath = "D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/feature_annotation_RTMS_neg_MSMSinjection.csv"
negtarget = pd.read_csv(negfilepath)

#select rows with not 'NotMatched'value in MSMS_injection column for each files
negtarget2 = negtarget[negtarget['MSMS_injection'] != 'NotMatched']
#convert the MSMS_injection column to int and then convert to str
negtarget2['MSMS_injection'] = negtarget2['MSMS_injection'].astype(int)
negtarget2['MSMS_injection'] = negtarget2['MSMS_injection'].astype(str)
negtarget2['MSMS_injection'] = 'B1-10-MSMS-neg_I' + negtarget2['MSMS_injection']


negtarget3 = negtarget[negtarget['MSMS_injection2'] != 'NotMatched']
negtarget3['MSMS_injection2'] = (
    negtarget3['MSMS_injection2']
      .astype(float)    # → 1.0, 2.0, …
      .astype(int)      # → 1, 2, …
      .astype(str))
negtarget3['MSMS_injection2'] = 'REVEAL_B1-10pool_Neg_Inj' + negtarget3['MSMS_injection2']

In [ ]:
#clean up the script and make filter on the number of matched peaks
##set threshold for peak matching
match_tolerance = 0.005 #dalton
peak_int_tol = 5 #ppm
peak_norm_tol = 5 #%

In [307]:
##import library for spectrum comparison
#perform spectrum search for each row by InChiKey_origin column, and add number of found spectra to the dataframe from each database.
#add tqdm progress bar to the loop

# suppress all matchms warnings
logging.getLogger("matchms").setLevel(logging.ERROR)

for iter, row in negtarget2.iterrows():
    key = row['InChiKey_origin']

    #extract the query spectrum from targeted injection
    query_spectrums =[]
    target_mz = row['Average Mz']
    injection_id = row['MSMS_injection']
    q_filepath = os.path.join("D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/NOV22/", f"{injection_id}.mzML")

    celevel = [10,20,40]
    for ce in celevel:
        ms2_query = preprocess_spectra(q_filepath, target_mz, ce, tolerance=peak_int_tol)
        if ms2_query is None:
            # print(f'No spectra found for compound {key} with CE {ce}')
            continue
        sspectrum = Spectrum(mz = normalize_and_filter(ms2_query[0], baseline=peak_norm_tol)[0],
                             intensities = normalize_and_filter(ms2_query[0], baseline=peak_norm_tol)[1], metadata = {"inchikey": key, 'peak_comments':  str(ce)+'eV'})
        query_spectrums.append(sspectrum)

    #extract the reference spectrum from database
    reference_spectrums_library = []
    reference_spectrums_insilico =[]

    sources = [
    (mbank_data_neg, reference_spectrums_library,    'mbank'),
    (mona_data_neg,  reference_spectrums_library,    'mona'),
    (mona_obtrap_data_neg, reference_spectrums_library, 'mona_obtrap'),
    (cfmid_e0_data_neg, reference_spectrums_insilico, 'cfmid_e0'),
    (cfmid_e1_data_neg, reference_spectrums_insilico, 'cfmid_e1'),
    (cfmid_e2_data_neg, reference_spectrums_insilico, 'cfmid_e2')]

    for data, library, prefix in sources:
        spectra_dict = get_spectrum_by_key(data, key)
        for ce, spectra in spectra_dict.items():
            for sspectrum in spectra:
                mz, intensities = normalize_and_filter(sspectrum, baseline=peak_norm_tol)
                library.append(
                    Spectrum(
                        mz=mz,
                        intensities=intensities,
                        metadata={
                            'inchikey': key,
                            'peak_comments': f"{prefix}_{ce}"
                        }
                    )
                )

    best_score_library, best_num_matchpeak_library, best_spectrum_library, best_match_query_library = find_best_match(
    reference_spectrums_library,
    query_spectrums,
    match_tolerance,
    key)

    best_score_insilico, best_num_matchpeak_insilico, best_spectrum_insilico, best_match_query_insilico = find_best_match(
        reference_spectrums_insilico,
        query_spectrums,
        match_tolerance,
        key)
    
    negtarget2.loc[iter, 'library_best_match'] = best_spectrum_library
    negtarget2.loc[iter, 'library_best_match_num_matchpeak'] = best_num_matchpeak_library
    negtarget2.loc[iter, 'library_best_match_query'] = best_match_query_library
    negtarget2.loc[iter, 'library_best_match_score_value'] = best_score_library
    negtarget2.loc[iter, 'insilico_best_match'] = best_spectrum_insilico
    negtarget2.loc[iter, 'insilico_best_match_num_matchpeak'] = best_num_matchpeak_insilico
    negtarget2.loc[iter, 'insilico_best_match_query'] = best_match_query_insilico
    negtarget2.loc[iter, 'insilico_best_match_score_value'] = best_score_insilico

In [70]:
##import library for spectrum comparison
#perform spectrum search for each row by InChiKey_origin column, and add number of found spectra to the dataframe from each database.
for iter, row in negtarget3.iterrows():
    key = row['InChiKey_origin']

    #extract the query spectrum from targeted injection
    query_spectrums =[]
    target_mz = row['Average Mz']
    injection_id = row['MSMS_injection2']
    q_filepath = os.path.join("D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/B1-10pool_tMSMS_04012025/", f"{injection_id}.mzML")


    celevel = [10,20,40]
    for ce in celevel:
        ms2_query = preprocess_spectra(q_filepath, target_mz, ce, tolerance=peak_int_tol)
        if ms2_query is None:
            # print(f'No spectra found for compound {key} with CE {ce}')
            continue
        sspectrum = Spectrum(mz = normalize_and_filter(ms2_query[0], baseline=peak_norm_tol)[0],
                             intensities = normalize_and_filter(ms2_query[0], baseline=peak_norm_tol)[1], metadata = {"inchikey": key, 'peak_comments':  str(ce)+'eV'})
        query_spectrums.append(sspectrum)

    #extract the reference spectrum from database
    reference_spectrums_library = []
    reference_spectrums_insilico =[]

    sources = [
    (mbank_data_neg, reference_spectrums_library,    'mbank'),
    (mona_data_neg,  reference_spectrums_library,    'mona'),
    (mona_obtrap_data_neg, reference_spectrums_library, 'mona_obtrap'),
    (cfmid_e0_data_neg, reference_spectrums_insilico, 'cfmid_e0'),
    (cfmid_e1_data_neg, reference_spectrums_insilico, 'cfmid_e1'),
    (cfmid_e2_data_neg, reference_spectrums_insilico, 'cfmid_e2')]

    for data, library, prefix in sources:
        spectra_dict = get_spectrum_by_key(data, key)
        for ce, spectra in spectra_dict.items():
            for sspectrum in spectra:
                mz, intensities = normalize_and_filter(sspectrum, baseline=peak_norm_tol)
                library.append(
                    Spectrum(
                        mz=mz,
                        intensities=intensities,
                        metadata={
                            'inchikey': key,
                            'peak_comments': f"{prefix}_{ce}"
                        }
                    )
                )

    best_score_library, best_num_matchpeak_library, best_spectrum_library, best_match_query_library = find_best_match(
    reference_spectrums_library,
    query_spectrums,
    match_tolerance,
    key)

    best_score_insilico, best_num_matchpeak_insilico, best_spectrum_insilico, best_match_query_insilico = find_best_match(
        reference_spectrums_insilico,
        query_spectrums,
        match_tolerance,
        key)
    
    negtarget3.loc[iter, 'library_best_match2'] = best_spectrum_library
    negtarget3.loc[iter, 'library_best_match_num_matchpeak2'] = best_num_matchpeak_library
    negtarget3.loc[iter, 'library_best_match_query2'] = best_match_query_library
    negtarget3.loc[iter, 'library_best_match_score_value2'] = best_score_library
    negtarget3.loc[iter, 'insilico_best_match2'] = best_spectrum_insilico
    negtarget3.loc[iter, 'insilico_best_match_num_matchpeak2'] = best_num_matchpeak_insilico
    negtarget3.loc[iter, 'insilico_best_match_query2'] = best_match_query_insilico
    negtarget3.loc[iter, 'insilico_best_match_score_value2'] = best_score_insilico

No match found for compound KHPXUQMNIQBQEV-UHFFFAOYSA-N
No match found for compound KHPXUQMNIQBQEV-UHFFFAOYSA-N
No match found for compound JFCQEDHGNNZCLN-UHFFFAOYSA-N
No match found for compound JFCQEDHGNNZCLN-UHFFFAOYSA-N
No match found for compound BPGDAMSIGCZZLK-UHFFFAOYSA-N
No match found for compound WOZHZOLFFPSEAM-UHFFFAOYSA-N
No match found for compound TYMLOMAKGOJONV-UHFFFAOYSA-N
No match found for compound PMDHMYFSRFZGIO-UHFFFAOYSA-N
No match found for compound HZHMMLIMOUNKCK-UHFFFAOYSA-N
No match found for compound RPBWMJBZQXCSFW-UHFFFAOYSA-N
No match found for compound ITHNIFCFNUZYLQ-UHFFFAOYSA-N
No match found for compound ABUZDQCQXDZFRG-UHFFFAOYSA-N
No match found for compound CTKMZCFTOKMBBS-UHFFFAOYSA-N
No match found for compound VYOCMAQYLOBQAS-UHFFFAOYSA-N
No match found for compound LGRFSURHDFAFJT-UHFFFAOYSA-N
No match found for compound XUKQTRYCLLDCIG-UHFFFAOYSA-N
No match found for compound LJYJFKXQKHSSEB-UHFFFAOYSA-N
No match found for compound POJGRKZMYVJCST-UHFFF

In [ ]:
#update the annotation_RTMS level for each row accroding to the spectrum matching results following the criteria below: 
# annotation level = NotMatched if RTMS = NotMatched
# annotation level 3 for library_best_match_score_value or library_best_match_score_value <0.5 and RTMS = 2 or RTMS =3 or RTMS = 4,
# annotation level 2b for insilico_best_match_score_value >= 0.5 and RTMS = 2 OR RTMS =3,
# annotation level 2a for library_best_match_score_value >= 0.5 and RTMS = 2 or RTMS =3, 
# annotation level = 5 for the rest of the rows
# Replace None values with 0 for comparison
negtarget2['library_best_match_score_value'].fillna(0, inplace=True)
negtarget2['insilico_best_match_score_value'].fillna(0, inplace=True)
negtarget3['library_best_match_score_value2'].fillna(0, inplace=True)
negtarget3['insilico_best_match_score_value2'].fillna(0, inplace=True)

#concatenate the two dataframe and make a new column as library_best_match_score_value and insilico_best_match_score_value
negtarget4 = pd.concat([negtarget2, negtarget3], axis=0, ignore_index=True)
negtarget4['library_best_match_score_value'] = negtarget4['library_best_match_score_value'].fillna(negtarget4['library_best_match_score_value2'])
negtarget4['insilico_best_match_score_value'] = negtarget4['insilico_best_match_score_value'].fillna(negtarget4['insilico_best_match_score_value2'])

#convert annoataion_RTMS to string
negtarget4['annotation_RTMS'] = negtarget4['annotation_RTMS'].astype(str)
negtarget4 = update_RTMSMS_level(negtarget4)

#count the number of rows with annotation_level == 2a
count_2a = len(negtarget4[negtarget4['annotation_RTMS'] == '2'])
print(f'The number of rows with RTMSMS_level == 2a: {count_2a}')

# negtarget4.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/Matched_peak_neg_spectrum_search_after_library_searching_20250804.csv", index=False)

In [362]:
negtarget4 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/Matched_peak_neg_spectrum_search_after_library_searching_20250804.csv")
negtarget4 = update_RTMSMS_level(negtarget4)

In [308]:
posfilepath = "D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/feature_annotation_RTMS_pos_MSMSinjection.csv"
postarget = pd.read_csv(posfilepath)

#select rows with not 'NotMatched'value in MSMS_injection column for each files
postarget2 = postarget[postarget['MSMS_injection'] != 'NotMatched']
postarget2['MSMS_injection'] = postarget2['MSMS_injection'].astype(str)
postarget2['MSMS_injection'] = 'B1-10-MSMS-pos_I' + postarget2['MSMS_injection']

logging.getLogger("matchms").setLevel(logging.ERROR)

for iter, row in postarget2.iterrows():
    key = row['InChiKey_origin']

    #extract the query spectrum from targeted injection
    query_spectrums =[]
    target_mz = row['Average Mz']
    injection_id = row['MSMS_injection']
    q_filepath = os.path.join("D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/NOV22/", f"{injection_id}.mzML")

    celevel = [10,20,40]
    for ce in celevel:
        ms2_query = preprocess_spectra(q_filepath, target_mz, ce, tolerance=peak_int_tol)
        if ms2_query is None:
            # print(f'No spectra found for compound {key} with CE {ce}')
            continue
        sspectrum = Spectrum(mz = normalize_and_filter(ms2_query[0], baseline=peak_norm_tol)[0],
                             intensities = normalize_and_filter(ms2_query[0], baseline=peak_norm_tol)[1], metadata = {"inchikey": key, 'peak_comments':  str(ce)+'eV'})
        query_spectrums.append(sspectrum)

    #extract the reference spectrum from database
    reference_spectrums_library = []
    reference_spectrums_insilico =[]

    sources = [
    (mbank_data_pos, reference_spectrums_library,    'mbank'),
    (mona_data_pos,  reference_spectrums_library,    'mona'),
    (mona_obtrap_data_pos, reference_spectrums_library, 'mona_obtrap'),
    (cfmid_e0_data_pos, reference_spectrums_insilico, 'cfmid_e0'),
    (cfmid_e1_data_pos, reference_spectrums_insilico, 'cfmid_e1'),
    (cfmid_e2_data_pos, reference_spectrums_insilico, 'cfmid_e2')]

    for data, library, prefix in sources:
        spectra_dict = get_spectrum_by_key(data, key)
        for ce, spectra in spectra_dict.items():
            for sspectrum in spectra:
                mz, intensities = normalize_and_filter(sspectrum, baseline=peak_norm_tol)
                library.append(
                    Spectrum(
                        mz=mz,
                        intensities=intensities,
                        metadata={
                            'inchikey': key,
                            'peak_comments': f"{prefix}_{ce}"
                        }
                    )
                )

    best_score_library, best_num_matchpeak_library, best_spectrum_library, best_match_query_library = find_best_match(
    reference_spectrums_library,
    query_spectrums,
    match_tolerance,
    key)

    best_score_insilico, best_num_matchpeak_insilico, best_spectrum_insilico, best_match_query_insilico = find_best_match(
        reference_spectrums_insilico,
        query_spectrums,
        match_tolerance,
        key)
    
    postarget2.loc[iter, 'library_best_match'] = best_spectrum_library
    postarget2.loc[iter, 'library_best_match_num_matchpeak'] = best_num_matchpeak_library
    postarget2.loc[iter, 'library_best_match_query'] = best_match_query_library
    postarget2.loc[iter, 'library_best_match_score_value'] = best_score_library
    postarget2.loc[iter, 'insilico_best_match'] = best_spectrum_insilico
    postarget2.loc[iter, 'insilico_best_match_num_matchpeak'] = best_num_matchpeak_insilico
    postarget2.loc[iter, 'insilico_best_match_query'] = best_match_query_insilico
    postarget2.loc[iter, 'insilico_best_match_score_value'] = best_score_insilico

In [309]:
posfilepath = "D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/feature_annotation_RTMS_pos_MSMSinjection.csv"
postarget = pd.read_csv(posfilepath)
postarget3 = postarget[postarget['MSMS_injection2'] != 'NotMatched']
postarget3['MSMS_injection2'] = postarget3['MSMS_injection2'].astype(float).astype(int).astype(str)
postarget3['MSMS_injection2'] = 'REVEAL_B1-10pool_Pos_Inj' + postarget3['MSMS_injection2']


logging.getLogger("matchms").setLevel(logging.ERROR)

for iter, row in postarget3.iterrows():
    key = row['InChiKey_origin']

    #extract the query spectrum from targeted injection
    query_spectrums =[]
    target_mz = row['Average Mz']
    injection_id = row['MSMS_injection2']
    q_filepath = os.path.join("D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/B1-10pool_tMSMS_04012025/", f"{injection_id}.mzML")

    celevel = [10,20,40]
    for ce in celevel:
        ms2_query = preprocess_spectra(q_filepath, target_mz, ce, tolerance=peak_int_tol)
        if ms2_query is None:
            # print(f'No spectra found for compound {key} with CE {ce}')
            continue
        sspectrum = Spectrum(mz = normalize_and_filter(ms2_query[0], baseline=peak_norm_tol)[0],
                             intensities = normalize_and_filter(ms2_query[0], baseline=peak_norm_tol)[1], metadata = {"inchikey": key, 'peak_comments':  str(ce)+'eV'})
        query_spectrums.append(sspectrum)

    #extract the reference spectrum from database
    reference_spectrums_library = []
    reference_spectrums_insilico =[]

    sources = [
    (mbank_data_pos, reference_spectrums_library,    'mbank'),
    (mona_data_pos,  reference_spectrums_library,    'mona'),
    (mona_obtrap_data_pos, reference_spectrums_library, 'mona_obtrap'),
    (cfmid_e0_data_pos, reference_spectrums_insilico, 'cfmid_e0'),
    (cfmid_e1_data_pos, reference_spectrums_insilico, 'cfmid_e1'),
    (cfmid_e2_data_pos, reference_spectrums_insilico, 'cfmid_e2')]

    for data, library, prefix in sources:
        spectra_dict = get_spectrum_by_key(data, key)
        for ce, spectra in spectra_dict.items():
            for sspectrum in spectra:
                mz, intensities = normalize_and_filter(sspectrum, baseline=peak_norm_tol)
                library.append(
                    Spectrum(
                        mz=mz,
                        intensities=intensities,
                        metadata={
                            'inchikey': key,
                            'peak_comments': f"{prefix}_{ce}"
                        }
                    )
                )

    best_score_library, best_num_matchpeak_library, best_spectrum_library, best_match_query_library = find_best_match(
    reference_spectrums_library,
    query_spectrums,
    match_tolerance,
    key)

    best_score_insilico, best_num_matchpeak_insilico, best_spectrum_insilico, best_match_query_insilico = find_best_match(
        reference_spectrums_insilico,
        query_spectrums,
        match_tolerance,
        key)
    
    postarget3.loc[iter, 'library_best_match2'] = best_spectrum_library
    postarget3.loc[iter, 'library_best_match_num_matchpeak2'] = best_num_matchpeak_library
    postarget3.loc[iter, 'library_best_match_query2'] = best_match_query_library
    postarget3.loc[iter, 'library_best_match_score_value2'] = best_score_library
    postarget3.loc[iter, 'insilico_best_match2'] = best_spectrum_insilico
    postarget3.loc[iter, 'insilico_best_match_num_matchpeak2'] = best_num_matchpeak_insilico
    postarget3.loc[iter, 'insilico_best_match_query2'] = best_match_query_insilico
    postarget3.loc[iter, 'insilico_best_match_score_value2'] = best_score_insilico

In [364]:
postarget2['library_best_match_score_value'].fillna(0, inplace=True)
postarget2['insilico_best_match_score_value'].fillna(0, inplace=True)
postarget3['library_best_match_score_value2'].fillna(0, inplace=True)
postarget3['insilico_best_match_score_value2'].fillna(0, inplace=True)

#concatenate the two dataframe and make a new column as library_best_match_score_value and insilico_best_match_score_value
postarget4 = pd.concat([postarget2, postarget3], axis=0, ignore_index=True)
postarget4['library_best_match_score_value'] = postarget4['library_best_match_score_value'].fillna(postarget4['library_best_match_score_value2'])
postarget4['insilico_best_match_score_value'] = postarget4['insilico_best_match_score_value'].fillna(postarget4['insilico_best_match_score_value2'])

#convert annoataion_RTMS to string
postarget4['annotation_RTMS'] = postarget4['annotation_RTMS'].astype(str)
postarget5 = update_RTMSMS_level(postarget4)

postarget5.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/Matched_peak_pos_spectrum_search_after_library_searching_20250408.csv", index=False)

In [4]:
#re import pos and neg database for data combination
postarget5 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/Matched_peak_pos_spectrum_search_after_library_searching_20250408.csv")
negtarget5 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/Matched_peak_neg_spectrum_search_after_library_searching_20250804.csv")

In [8]:
logging.getLogger("matchms").setLevel(logging.ERROR)

#select targeted ms1 for spectrum matching
posfor_std = postarget5[postarget5['annotation_level'] == '2a']
##compare spectrum for row with 2a level and with the standard spectrum
posfor_std2 = remove_redundant_rows(posfor_std)
print(f'The number of unique rows with annotation_level == 2a: {len(posfor_std2)}')

#in house customized spectrum matching workflows
pos_entact = pd.read_csv('D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/for_msms/ENTACT_RT_pos_formsms.csv')
neg_entact = pd.read_csv('D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/for_msms/ENTACT_RT_neg_formsms.csv')

#get group_id from inquiry on the DTXSID in pos_entact and neg_entact
for iter, row in posfor_std2.iterrows():
    key = row['DTXSID']
    mz = row['Average Mz']
    rt = row['Average Rt(min)']

    #inquiry group_id according to dtxsid
    group_id = pos_entact[pos_entact['DTXSID'] == key]['group_id'].values[0]
    if group_id == None:
        print(f'No group_id found for compound {key}')
        continue

    #specify std spectrum path
    ref_filepath = os.path.join('D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/for_msms/mzml files/', f"{group_id}_100x_tMSMS_pos.mzML")

    #specify experimental spectrum path by taking none null value of row['MSMS_injection] and row['MSMS_injection2']
    if row['MSMS_injection'] != 'NotMatched':
        injection_id = row['MSMS_injection']
        q_filepath = os.path.join("D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/NOV22/", f"{injection_id}.mzML")

    elif row['MSMS_injection2'] != 'NotMatched':
        injection_id = row['MSMS_injection2']
        q_filepath = os.path.join("D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/B1-10pool_tMSMS_04012025/", f"{injection_id}.mzML")
    else:
        print(f'No experimental spectrum found for compound {key}')
        continue

    #do matching for each CE level and map each score to the corresponding CE level
    ce = [10,20,40]
    matchres = {}
    baseline = 5

    for ce in ce:
        #extract the experimental spectrum
        ms2_query = preprocess_spectra(q_filepath, mz, ce, tolerance=10)
        if ms2_query is None:
            print(f'No spectra found for compound {key} with CE {ce}')
            continue
        sspectrum = Spectrum(mz = normalize_and_filter(ms2_query[0], baseline=baseline)[0],
                             intensities = normalize_and_filter(ms2_query[0], baseline=baseline)[1], metadata = {"inchikey": key, 'peak_comments':  str(ce)+'eV'})
        
        #extract the reference spectrum
        ms2_ref = preprocess_spectra(ref_filepath, mz, ce, tolerance=10)
        reference = Spectrum(mz=normalize_and_filter(ms2_ref[0], baseline=baseline)[0], 
                            intensities = normalize_and_filter(ms2_ref[0], baseline=baseline)[1],
                            metadata={"inchikey": key})   

        #spectrum matching by cosine similarity
        # cosine_greedy = CosineGreedy(tolerance=0.001, mz_power=1.3, intensity_power=0.53)
        cosine_greedy = CosineGreedy(tolerance=0.001)
        score = cosine_greedy.pair(reference, sspectrum)
        score_value, matchpeaks= score['score'], score['matches']

        #create a dictionary to store the socre, matchpeaks and the corresponding CE level
        matchres[ce] = (score_value, matchpeaks)

    #find the highest score and the corresponding CE level
    best_score = 0.0
    query_best = None
    matched_numpeaks = 0.0

    for ce, (score_value, matchpeaks) in matchres.items():
        if matchpeaks >= 2.0:
            if score_value > best_score:
                best_score = score_value
                query_best = 'query' + '_' + str(ce)+'_eV'
                matched_numpeaks = matchpeaks
        else:
            continue
    
    posfor_std2.loc[iter, f'best_score'] = best_score
    posfor_std2.loc[iter, f'best_spectrum_query'] = query_best
    posfor_std2.loc[iter, f'best_match_num_peaks'] = matched_numpeaks

#entact_n_filepath = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/for_msms/mzml files/'+'E499_100x_tMSMS_neg.mzML'
#create a dataframe for the spectrum database and plot the spectrum

The number of unique rows with annotation_level == 2a: 6


In [9]:
#select rows with best_match_num_peaks >= 2.0 and best_score > 0.0
posfor_std3 = posfor_std2[(posfor_std2['best_match_num_peaks'] >= 2.0) & (posfor_std2['best_score'] > 0.0)]
print(tabulate.tabulate(posfor_std3[['feature_id','PREFERRED_NAME','DTXSID','InChiKey_origin', 'best_score', 'best_spectrum_query', 'best_match_num_peaks']], headers='keys', tablefmt='psql'))

+----+------------------+-------------------------+---------------+-----------------------------+--------------+-----------------------+------------------------+
|    | feature_id       | PREFERRED_NAME          | DTXSID        | InChiKey_origin             |   best_score | best_spectrum_query   |   best_match_num_peaks |
|----+------------------+-------------------------+---------------+-----------------------------+--------------+-----------------------+------------------------|
|  0 | 4.661_212.11842  | 1,3-Diphenylguanidine   | DTXSID3025178 | OWRCNXZUPFZXOS-UHFFFAOYSA-N |     0.803689 | query_20_eV           |                      4 |
|  1 | 5.195_146.05991  | 8-Hydroxyquinoline      | DTXSID5020730 | MCJGNVYPOGVAJF-UHFFFAOYSA-N |     0.548975 | query_40_eV           |                      6 |
|  2 | 11.783_315.23196 | Progesterone            | DTXSID3022370 | RJKFOVLPORLFTN-LEKSSAKUSA-N |     0.982831 | query_20_eV           |                      4 |
|  3 | 11.373_198.18484 | 1-

In [10]:
logging.getLogger("matchms").setLevel(logging.ERROR)

##compare spectrum for row with 2a level and with the standard spectrum
negfor_std = negtarget5[(negtarget5['annotation_level'] == '2a')|(negtarget5['annotation_level'] == '1')]
#remove redundant rows and select feature id for spectrum matching with analytical standards, #level 2a
negfor_std2 = remove_redundant_rows(negfor_std)
print(f'The number of unique rows with annotation_level == 2a: {len(negfor_std2)}')


#get group_id from inquiry on the DTXSID in pos_entact and neg_entact
for iter, row in negfor_std2.iterrows():
    key = row['DTXSID']
    mz = row['Average Mz']
    rt = row['Average Rt(min)']

    if key == 'DTXSID8031865' or key == 'DTXSID3031864':
        ref_filepath = os.path.join('D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/for_msms/mzml files/', 'REVEAL_PFAS-TarMSMS.mzML')
    else:
        #inquiry group_id according to dtxsid
        try: 
            group_id = neg_entact[neg_entact['DTXSID'] == key]['group_id'].values[0]

        except IndexError:
            group_id = None
            print(f'No group_id found for compound {key}')
            continue

        #specify std spectrum path
        ref_filepath = os.path.join('D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/for_msms/mzml files/', f"{group_id}_100x_tMSMS_neg.mzML")

    #specify experimental spectrum path by taking none null value of row['MSMS_injection] and row['MSMS_injection2']
    if row['MSMS_injection'] != 'NotMatched':
        injection_id = row['MSMS_injection']
        q_filepath = os.path.join("D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/NOV22/", f"{injection_id}.mzML")

    elif row['MSMS_injection2'] != 'NotMatched':
        injection_id = row['MSMS_injection2']
        q_filepath = os.path.join("D:/UCSF_postdoc_topic/REVEAL_topics/R200_MSMS/B1-10pool_tMSMS_04012025/", f"{injection_id}.mzML")
    else:
        print(f'No experimental spectrum found for compound {key}')
        continue

    ce = [10,20,40]
    best_score = 0
    baseline = 0.5
    #do matching for each CE level and map each score to the corresponding CE level
    for ce in ce:
        #extract the experimental spectrum
        ms2_query = preprocess_spectra(q_filepath, mz, ce, tolerance=10)
        if ms2_query is None:
            print(f'No spectra found for compound {key} with CE {ce}')
            continue
    
        if len(normalize_and_filter(ms2_query[0], baseline=baseline)[0]) <= 1 or len(normalize_and_filter(ms2_ref[0], baseline=baseline)[0]) <= 1:
            print(f'no more than 1 spectrum in ref or query found for compound {key} with CE {ce}')
            continue
        sspectrum = Spectrum(mz = normalize_and_filter(ms2_query[0], baseline=baseline)[0],
                             intensities = normalize_and_filter(ms2_query[0], baseline=baseline)[1], metadata = {"inchikey": key, 'peak_comments':  str(ce)+'eV'})
        
        #extract the reference spectrum
        ms2_ref = preprocess_spectra(ref_filepath, mz, ce, tolerance=10)
        reference = Spectrum(mz=normalize_and_filter(ms2_ref[0], baseline=baseline)[0], 
                            intensities = normalize_and_filter(ms2_ref[0], baseline=baseline)[1],
                            metadata={"inchikey": key})   

        #spectrum matching by cosine similarity
        # cosine_greedy = CosineGreedy(tolerance=0.005, mz_power=1.3, intensity_power=0.53)
        cosine_greedy = CosineGreedy(tolerance=0.001)
        score = cosine_greedy.pair(reference, sspectrum)
        score_value, matchpeaks= score['score'], score['matches']

        #create a dictionary to store the socre, matchpeaks and the corresponding CE level
        matchres[ce] = (score_value, matchpeaks)

    #find the highest score and the corresponding CE level
    best_score = 0.0
    query_best = None
    matched_numpeaks = 0.0

    for ce, (score_value, matchpeaks) in matchres.items():
        if matchpeaks >= 2.0:
            if score_value > best_score:
                best_score = score_value
                query_best = 'query' + '_' + str(ce)+'_eV'
                matched_numpeaks = matchpeaks
        else:
            continue
    
    negfor_std2.loc[iter, f'best_score'] = best_score
    negfor_std2.loc[iter, f'best_spectrum_query'] = query_best
    negfor_std2.loc[iter, f'best_match_num_peaks'] = matched_numpeaks

The number of unique rows with annotation_level == 2a: 5


In [12]:
#select rows with best_match_num_peaks >= 2.0 and best_score > 0.0
negfor_std3 = negfor_std2[(negfor_std2['best_match_num_peaks'] >= 2.0) & (negfor_std2['best_score'] > 0.0)]
print(tabulate.tabulate(negfor_std3[['feature_id','PREFERRED_NAME','DTXSID','InChiKey_origin', 'best_score', 'best_spectrum_query', 'best_match_num_peaks']], headers='keys', tablefmt='psql'))

+----+------------------+------------------------------+---------------+-----------------------------+--------------+-----------------------+------------------------+
|    | feature_id       | PREFERRED_NAME               | DTXSID        | InChiKey_origin             |   best_score | best_spectrum_query   |   best_match_num_peaks |
|----+------------------+------------------------------+---------------+-----------------------------+--------------+-----------------------+------------------------|
|  0 | 9.72_412.9668    | Perfluorooctanoic acid       | DTXSID8031865 | SNGREZUHAYWORS-UHFFFAOYSA-N |     0.868809 | query_20_eV           |                      2 |
|  1 | 10.526_498.93045 | Perfluorooctanesulfonic acid | DTXSID3031864 | YFSUTJLHUFNCNZ-UHFFFAOYSA-N |     0.99964  | query_10_eV           |                      3 |
|  2 | 4.073_215.12936  | Undecanedioic acid           | DTXSID0044862 | LWBHHRRTOZQPDM-UHFFFAOYSA-N |     0.942635 | query_10_eV           |                      3 

In [15]:
#update annotation level based on the spectrum matching, add best score to the neg_target5 and pos_target5 dataframe
#add best_score and best_spectrum to pos_target5 and neg_target5 dataframe according to the feature_id and DTXSID
import pandas as pd
postarget5 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/Matched_peak_pos_spectrum_search_after_library_searching_20250408.csv")
negtarget5 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/Matched_peak_neg_spectrum_search_after_library_searching_20250804.csv")

postarget5 = pd.merge(postarget5, posfor_std2[['feature_id', 'DTXSID','best_score', 'best_spectrum_query','best_match_num_peaks']], on=['feature_id','DTXSID'], how='left')
negtarget5 = pd.merge(negtarget5, negfor_std2[['feature_id', 'DTXSID','best_score', 'best_spectrum_query','best_match_num_peaks']], on=['feature_id','DTXSID'], how='left')


postarget5 = update_RTMSMS_level2(postarget5)
negtarget5 = update_RTMSMS_level2(negtarget5)

# postarget5.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/Matched_peak_pos_spectrum_search_after_library_searching_20250508.csv", index=False)
# negtarget5.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/Matched_peak_neg_spectrum_search_after_library_searching_20250508.csv", index=False)

#print the number of rows with annotation_level == 'NotMatched' in postarget5 and negtarget
print(f'The number of rows with annotation_level == NotMatched in postarget5 and neg: {len(postarget5[postarget5["annotation_level"] == "NotMatched"]) +len(negtarget5[negtarget5["annotation_level"] == "NotMatched"])}')
postarget5_notmatched = postarget5[postarget5['annotation_level'] == 'NotMatched']
negtarget5_notmatched = negtarget5[negtarget5['annotation_level'] == 'NotMatched']
#combined the two dataframme and find the unique DTXSID 
target5_notmatched = pd.concat([postarget5_notmatched, negtarget5_notmatched], axis=0, ignore_index=True)
#print the unique number of DTXSID in the dataframe
print(f'The number of unique DTXSID in target5_notmatched: {len(target5_notmatched["DTXSID"].unique())}')

#remove rows with annotation_level == 'NotMatched'
postarget5 = postarget5[(postarget5['annotation_level'] != 'NotMatched')]
negtarget5 = negtarget5[(negtarget5['annotation_level'] != 'NotMatched')]

#summary of the annotation level value for postarget5
print(postarget5['annotation_level'].value_counts())
#summary of the annotation level value for negtarget5
print(negtarget5['annotation_level'].value_counts())

The number of rows with annotation_level == NotMatched in postarget5 and neg: 154
The number of unique DTXSID in target5_notmatched: 41
annotation_level
5     1415
2b     329
4      170
3       80
1        6
2a       2
Name: count, dtype: int64
annotation_level
5     2204
2b    1304
4      344
3      121
1        3
2a       1
Name: count, dtype: int64


In [440]:
#print the number of unique feature_id with annotation level == (2a or 1, or 2b) in total, calculate total number = postarget5 + negtarget5
print(f'The number of unique feature_id with annotation level == (2a or 1, or 2b) in total: {len(postarget5[postarget5["annotation_level"].isin(["2a", "1", "2b"])]) + len(negtarget5[negtarget5["annotation_level"].isin(["2a", "1", "2b"])])}')

#print the number of unique feature_id with annotation level == 3 in total
print(f'The number of unique feature_id with annotation level == 3 in total: {len(postarget5[postarget5["annotation_level"] == "3"]) + len(negtarget5[negtarget5["annotation_level"] == "3"])}')

#print the number of unique feature_id with annotation level == 4 in total
print(f'The number of unique feature_id with annotation level == 4 in total: {len(postarget5[postarget5["annotation_level"] == "4"]) + len(negtarget5[negtarget5["annotation_level"] == "4"])}')


The number of unique feature_id with annotation level == (2a or 1, or 2b) in total: 1645
The number of unique feature_id with annotation level == 3 in total: 201
The number of unique feature_id with annotation level == 4 in total: 514


In [ ]:
#subset rows with annotation level 1, 2a, 2b.
postarget6 = postarget5[(postarget5['annotation_level'] == '1') | (postarget5['annotation_level'] == '2a') | (postarget5['annotation_level'] == '2b')]
negtarget6 = negtarget5[(negtarget5['annotation_level'] == '1') | (negtarget5['annotation_level'] == '2a') | (negtarget5['annotation_level'] == '2b')]

##remove the redundant rows in postarget6 and negtarget6
postarget8 = remove_redundant_rows2(postarget6)
negtarget8 = remove_redundant_rows2(negtarget6)
print(f'The number of unique rows with annotation level 1, 2a, 2b in pos_target8: {len(postarget8)}')
print(f'The number of unique rows with annotation level 1, 2a, 2b in neg_target8: {len(negtarget8)}')
#unique DTXSID in postarget8 and negtarget8
print(f'The number of unique DTXSID in pos_target8: {len(postarget8["DTXSID"].unique())}')
print(f'The number of unique DTXSID in neg_target8: {len(negtarget8["DTXSID"].unique())}')
#print unique feature_id in postarget8 and negtarget8
print(f'The number of unique feature_id in pos_target8: {len(postarget8["feature_id"].unique())}')
print(f'The number of unique feature_id in neg_target8: {len(negtarget8["feature_id"].unique())}')

# # ##output dataframe
postarget8.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/library_search_res_pos_level1andlevel2.csv')
negtarget8.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/library_search_res_neg_level1andlevel2.csv')

The number of unique rows with annotation level 1, 2a, 2b in pos_target8: 240
The number of unique rows with annotation level 1, 2a, 2b in neg_target8: 799
The number of unique DTXSID in pos_target8: 159
The number of unique DTXSID in neg_target8: 307
The number of unique feature_id in pos_target8: 165
The number of unique feature_id in neg_target8: 392


In [434]:
#print and concat the level 1 annotations 
identifications = pd.concat([postarget8[postarget8["annotation_level"] == '1'],negtarget8[negtarget8["annotation_level"] == '1']], axis=0, ignore_index=True)
print(tabulate.tabulate(identifications, headers='keys', tablefmt='pretty'))
#output identifications
identifications.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/level1_annotations_pos_neg_20250428.csv", index=False)

+---+------------+-----------------+------------+--------------------------------------------------------------------------------+------------------------------+---------------+---------------+-----------------------------+----------------------------+------------------+---------------------------------------------------------------------+-------------------------+-------------+-------------------+-------------------+------------+-------------+-------------------+------------------------+-----------------+------------------------+------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+---------------+------------------+-------------------+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-

In [ ]:
#calculate the detection frequency of feature id at level 1 annotation, and plot heatmap
# postarget7 = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/library_search_res_pos_level1andlevel2.csv')
poslevel1= postarget8[postarget8['annotation_level'] == '1']
feature_id = poslevel1['feature_id']
compound_name = poslevel1['PREFERRED_NAME']
#extract column starts with 'BH' and 'feature_id'
poslevel1 = poslevel1.loc[:, poslevel1.columns.str.startswith('BH')]
#convert column value to float, except feature_id column
poslevel1 = poslevel1.astype(float)
#calculate the detection frequency: (number of BH column with the value >=5000 / total number of BH columns) * 100, add the detection frequency to the dataframe and named as 'detection_frequency'
poslevel1['detection_frequency'] = poslevel1.iloc[:, 1:].apply(lambda x: (x >= 5000).sum() / len(x) * 100, axis=1)
poslevel1['feature_id'] = feature_id
poslevel1['Matched_compound'] = compound_name
#sort the dataframe by detection frequency in descending order
poslevel1 = poslevel1.sort_values(by='detection_frequency', ascending=False)
print(tabulate.tabulate(poslevel1[['detection_frequency','feature_id','Matched_compound']], headers='keys', tablefmt='psql'))

+-----+-----------------------+------------------+-------------------------+
|     |   detection_frequency | feature_id       | Matched_compound        |
|-----+-----------------------+------------------+-------------------------|
| 142 |              100      | 13.158_279.15945 | Diisobutyl phthalate    |
|   3 |               93.4673 | 5.195_146.05991  | 8-Hydroxyquinoline      |
|  23 |               69.8492 | 11.783_315.23196 | Progesterone            |
| 147 |               69.3467 | 14.529_403.23285 | Acetyl tributyl citrate |
| 130 |               51.7588 | 11.373_198.18484 | 1-Octyl-2-pyrrolidone   |
|   1 |               42.2111 | 4.661_212.11842  | 1,3-Diphenylguanidine   |
+-----+-----------------------+------------------+-------------------------+


In [ ]:
neglevel1= negtarget8[negtarget8['annotation_level'] == '1']
feature_id = neglevel1['feature_id']
compound_name = neglevel1['PREFERRED_NAME']
#extract column starts with 'BH' and 'feature_id'
neglevel1 = neglevel1.loc[:, neglevel1.columns.str.startswith('BH')]
#convert column value to float, except feature_id column
neglevel1 = neglevel1.astype(float)
#calculate the detection frequency: (number of BH column with the value >=5000 / total number of BH columns) * 100, add the detection frequency to the dataframe and named as 'detection_frequency'
neglevel1['detection_frequency'] = neglevel1.iloc[:, 1:].apply(lambda x: (x >= 5000).sum() / len(x) * 100, axis=1)
neglevel1['feature_id'] = feature_id
neglevel1['Matched_compound'] = compound_name
#sort the dataframe by detection frequency in descending order
neglevel1 = neglevel1.sort_values(by='detection_frequency', ascending=False)
print(tabulate.tabulate(neglevel1[['detection_frequency','feature_id','Matched_compound']], headers='keys', tablefmt='psql'))

+-----+-----------------------+------------------+------------------------------+
|     |   detection_frequency | feature_id       | Matched_compound             |
|-----+-----------------------+------------------+------------------------------|
|  39 |                  85.5 | 10.526_498.93045 | Perfluorooctanesulfonic acid |
| 131 |                  74.5 | 4.073_215.12936  | Undecanedioic acid           |
|  30 |                  39   | 9.72_412.9668    | Perfluorooctanoic acid       |
+-----+-----------------------+------------------+------------------------------+


In [ ]:
#import hmdb metabolite list...
